## Inštalácia YOLO-NAS


In [ ]:
!pip install -q git+https://github.com/Deci-AI/super-gradients.git@stable
!pip install -q roboflow
!pip install -q supervision

**Poznámka:** Aby sme si uľahčili prácu so súbormi údajov, obrázkami a modelmi, vytvoríme konštantu `HOME`.

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


Tento kód overuje dostupnosť GPU a nastavuje zariadenie na 'cuda', ak je k dispozícii. Následne definuje architektúru modelu YOLO-NAS 'yolo_nas_l'.

In [3]:
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
MODEL_ARCH = 'yolo_nas_l'

In [4]:
f"{HOME}/data"

'/content/data'

Nastavenie cesty k súboru údajov pomocou premennej `LOCATION` a tried súboru údajov pomocou premennej `CLASSES`

In [ ]:
LOCATION = f"Meteory_format_YOLOv5_aug"
print("location:", LOCATION)
CLASSES = ['meteor', 'satelite']
print("classes:", CLASSES)

## Príprava dát do vhodného formátu
`ckpt_root_dir` - toto je adresár, do ktorého sa budú ukladať výsledky všetkých  experimentov

`experiment_name` - názov experimentu

In [11]:
BATCH_SIZE = 16
MAX_EPOCHS = 200
CHECKPOINT_DIR = f'yolo_nas_checkpoints'
EXPERIMENT_NAME = f'yolo_nas_models'

In [ ]:
from super_gradients.training import Trainer

trainer = Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

Definovanie ciest k tréningovým, validačným a testovacím súborom.

In [ ]:
dataset_params = {
    'data_dir': LOCATION,
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': CLASSES
}

Príprava údajov na trénovanie, validáciu a testovanie modelu detekcie objektov na základe súboru údajov vo formáte kompatibilnom s YOLO.

In [ ]:
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

### Vytvorenie inštancie modelu

In [ ]:
from super_gradients.training import models
import supervision as sv
import cv2

model = models.get(
    MODEL_ARCH,
    num_classes=len(dataset_params['classes']),
    pretrained_weights="coco"
)


`checkpoint_path` - nastavenie cesty ku kontrolnému bodu, v prípade, že začíname trénovať s predtrénovaným modelom

In [ ]:
from super_gradients.training import models
import supervision as sv
import cv2

checkpoint_path = 'model/model_yolonas.pth'
checkpoint = torch.load(checkpoint_path, map_location=DEVICE)

model_weights = checkpoint['net']

model = models.get(MODEL_ARCH, num_classes=2, pretrained_weights=None).to(DEVICE)
model.load_state_dict(model_weights)

### Definovanie metrík, nastavení a cieľových metrík

In [ ]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "SGD",
    "optimizer_params": {"weight_decay": 0.0001, "lr": 0.001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

### Spustenie tréningu modelu YOLO-NAS

In [ ]:
trainer.train(
    model=model,
    training_params=train_params,
    train_loader=train_data,
    valid_loader=val_data
)

## Načítanie natrénovaného modelu

`checkpoint_path` - cesta k hotovému modelu


In [ ]:
best_model = models.get(
    'yolo_nas_l',
    num_classes=len(dataset_params['classes']),
    checkpoint_path=f"model/model_yolonas.pth"
).to(DEVICE)

## Validácia modelu

In [ ]:
trainer.test(
    model=best_model,
    test_loader=val_data,
    test_metrics_list=DetectionMetrics_050(
        score_thres=0.1,
        top_k_predictions=300,
        num_cls=len(dataset_params['classes']),
        normalize_targets=True,
        post_prediction_callback=PPYoloEPostPredictionCallback(
            score_threshold=0.01,
            nms_top_k=1000,
            max_predictions=300,
            nms_threshold=0.7
        )
    )
)

### Inferencia s natrénovaným modelom

In [ ]:
import supervision as sv

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{LOCATION}/test/images",
    annotations_directory_path=f"{LOCATION}/test/labels",
    data_yaml_path=f"{LOCATION}/data.yaml",
    force_masks=False
)

In [ ]:
import supervision as sv

CONFIDENCE_THRESHOLD = 0.4

predictions = {}

for image_name, image in ds.images.items():
    result = best_model.predict(image, conf=CONFIDENCE_THRESHOLD)

    # Access prediction information from the ImageDetectionPrediction object
    bboxes_xyxy = result.prediction.bboxes_xyxy
    confidence = result.prediction.confidence
    labels = result.prediction.labels.astype(int)

    detections = sv.Detections(
        xyxy=bboxes_xyxy,
        confidence=confidence,
        class_id=labels
    )

    predictions[image_name] = detections


In [ ]:
import random
random.seed(10)

In [ ]:
import supervision as sv

MAX_IMAGE_COUNT = 241

n = min(MAX_IMAGE_COUNT, len(ds.images))

keys = list(ds.images.keys())
keys = random.sample(keys, n)

box_annotator = sv.BoxAnnotator()

images = []
titles = []

for key in keys:
    frame_with_annotations = box_annotator.annotate(
        scene=ds.images[key].copy(),
        detections=ds.annotations[key],
        skip_label=True
    )
    images.append(frame_with_annotations)
    titles.append('annotations')
    frame_with_predictions = box_annotator.annotate(
        scene=ds.images[key].copy(),
        detections=predictions[key],
        skip_label=True
    )
    images.append(frame_with_predictions)
    titles.append('predictions')

%matplotlib inline
grid_size = (n, 2) 
sv.plot_images_grid(images=images, titles=titles, grid_size=grid_size, size=(2 * 4, n * 2.5))
